attempting to run an autoML pipeline for location A

In [1]:
from tpot import TPOTRegressor
from functions import load_data, get_train_targets, get_test_data, prepare_submission
from sklearn.model_selection import train_test_split, RepeatedKFold

data_a, data_b, data_c = load_data()

X_train_a, targets_a = get_train_targets(data_a)

In [2]:
X_train_a.columns

Index(['time', 'absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_sp

In [3]:
drop_cols = ['time', 'date_calc', 'elevation:m', 'fresh_snow_1h:cm', 'wind_speed_u_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']

In [13]:
X_train_a['super_cooled_liquid_water:kgm2'].describe()

count    34061.000000
mean         0.055703
std          0.109655
min          0.000000
25%          0.000000
50%          0.000000
75%          0.100000
max          1.400000
Name: super_cooled_liquid_water:kgm2, dtype: float64

In [12]:
X = X_train_a.drop(columns=drop_cols).fillna(0)
y = targets_a

cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)

model = TPOTRegressor(generations=5, population_size=50, scoring='neg_mean_absolute_error', cv=cv, verbosity=2, random_state=1, n_jobs=-1)
model.fit(X, y)
model.export('tpot_locA_best_model.py')

                                                                               
                                                                               
TPOT closed during evaluation in one generation.
                                                                               
                                                                               
TPOT closed prematurely. Will use the current best pipeline.
                                                                               
Best pipeline: XGBRegressor(ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.3, min_samples_leaf=1, min_samples_split=12, n_estimators=100), learning_rate=1.0, max_depth=10, min_child_weight=2, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.7500000000000001, verbosity=0)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_train_a.drop(columns=drop_cols).fillna(0), targets_a,
                                                     train_size=0.9, test_size=0.1, random_state=42)
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_loc_A_pipeline.py')

                                                                               
                                                                               
TPOT closed during evaluation in one generation.
                                                                               
                                                                               
TPOT closed prematurely. Will use the current best pipeline.


- ran using the [autoML_run.py](autoML_run.py) script

## Using the output from the autoML_run script to generate submission:

In [8]:
import pandas as pd
import numpy as np
from functions import load_data, get_train_targets, get_test_data, prepare_submission, remove_ouliers

data_a, data_b, data_c = load_data()

data_a = remove_ouliers(data_a)
data_b = remove_ouliers(data_b)
data_c = remove_ouliers(data_c)

X_train_a, targets_a = get_train_targets(data_a)
X_train_b, targets_b = get_train_targets(data_b)
X_train_c, targets_c = get_train_targets(data_c)

X_test_a, X_test_b, X_test_c = get_test_data()

In [10]:
drop_cols = ['time', 'date_calc', 'elevation:m', 'fresh_snow_1h:cm', 'wind_speed_u_10m:ms', 
             'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms', 'prob_rime:p',
             'fresh_snow_12h:cm','fresh_snow_24h:cm', 'fresh_snow_6h:cm', 'super_cooled_liquid_water:kgm2']

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drops columns from the data."""

    def __init__(self, drop_cols = []):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        return X_copy.drop(columns=self.drop_cols)

data_process_pipeline = Pipeline([
    ('drop_cols', ColumnDropper(drop_cols=drop_cols)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
])

locA_pipeline = Pipeline([
    ('data_process', data_process_pipeline),
    ('random_forest', XGBRegressor(learning_rate=0.1, max_depth=9, min_child_weight=6, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.7000000000000001, verbosity=0))
])

locB_pipeline = Pipeline([
    ('data_process', data_process_pipeline),
    ('random_forest', RandomForestRegressor(bootstrap=False, max_features=0.4, min_samples_leaf=5, min_samples_split=2, n_estimators=100, random_state=1))
])

locC_pipeline = Pipeline([
    ('data_process', data_process_pipeline),
    ('random_forest', XGBRegressor(learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.8500000000000001, verbosity=0))
])

In [12]:
locA_pipeline.fit(X_train_a, targets_a)
pred_a = locA_pipeline.predict(X_test_a.drop(columns=["id", "prediction", "location"]))

In [13]:
locB_pipeline.fit(X_train_b, targets_b)
pred_b = locB_pipeline.predict(X_test_b.drop(columns=["id", "prediction", "location"]))

In [14]:
locC_pipeline.fit(X_train_c, targets_c)
pred_c = locC_pipeline.predict(X_test_c.drop(columns=["id", "prediction", "location"]))

In [15]:
submission = prepare_submission(X_test_a, X_test_b, X_test_c, pred_a, pred_b, pred_c)

In [20]:
submission.describe()

,id,prediction
count,2160.000000,2160.000000
mean,1079.500000,559.075300
std,623.682612,954.108086
min,0.000000,0.000000
25%,539.750000,17.031398
50%,1079.500000,198.923191
75%,1619.250000,594.383620
max,2159.000000,5197.416504


In [19]:
submission['prediction'] = submission['prediction'].apply(lambda x: 0 if x < 0.05 else x)

In [21]:
submission.to_csv('submissions/autoML_outlier_removal.csv', index=False)